In [215]:
# Return an indefinite article "a" or "an" for the animal name.
def article(name):
    if name[0].lower() in 'aeiou':
        return 'an'
    else:
        return 'a'

In [216]:
from __future__ import annotations
from enum import Enum
from functools import total_ordering

class Truth(Enum):
    FALSE = 0
    TRUE = 1
    UNKNOWN = 2

    # Return TRUE, FALSE, or UNKNOWN.
    def __str__(self):
        return self.name

class Operator(Enum):
    NONE = 5
    FACT = 4
    PAREN = 3
    NOT = 2
    AND = 1
    OR = 0

# Operator precedence.
# A higher number means a greater precedence.
class Precedence(Enum):
    NONE = 5
    FACT = 4
    PAREN = 3
    NOT = 2
    AND = 1
    OR = 1
    def __lt__(self, other: Precedence):
        if self.__class__ is other.__class__:
            return self.value < other.value
        return NotImplemented

# Return the enum values for the operator and its precedence.
def parse_operator(operator):
    if operator == '!':
        return Operator.NOT, Precedence.NOT
    if operator == '&':
        return Operator.AND, Precedence.AND
    if operator == '|':
        return Operator.OR, Precedence.OR
    return Operator.NONE, Precedence.NONE

In [217]:
from __future__ import annotations
from typing import NamedTuple, cast

# Represents a complete rule of the form 'antecedent -> consequent'.
# The antecedent is a BoolExpression and the consequent is a fact.
class Rule:
    
    _truth: Truth
    rule_string: str
    antecedent_string: str
    consequent_string: str
    is_negated: bool
    is_goal: bool

    antecedent: BoolExpression
    consequent: str
    def __init__(self, rule_string: str):
        self.antecedent_truth = Truth.UNKNOWN
        self.rule_string = rule_string

        rule_parts = rule_string.split('->')
        self.antecedent_string = rule_parts[0].strip()
        self.consequent_string = rule_parts[1].strip()
        self.is_negated = self.consequent_string.startswith('!')
        self.is_goal = self.consequent_string.startswith('$')
        self.consequent = self.consequent_string.lstrip('!').lstrip('$').strip()
        self.antecedent = BoolExpression(self.antecedent_string)

    # Return the truth of the rule's consequent.
    def truth(self, facts):
        return facts[self.consequent]

    # Return a set holding this rule's facts. That includes
    # the consequent and the facts in the antecedent.
    def list_facts(self):
        return self.antecedent.list_facts().union({self.consequent})

    # Evaluate the antecedent. If it's now TRUE,
    # then set the consequent's value.
    def evaluate(self, facts: dict[str, Truth], explain: bool):
        if facts[self.consequent] != Truth.UNKNOWN:
            return facts[self.consequent]
        
        if self.antecedent_truth != Truth.UNKNOWN:
            return facts[self.consequent]

        truth = self.antecedent.evaluate(facts)

        if truth == Truth.TRUE:
            facts[self.consequent] = Truth.FALSE if self.is_negated else Truth.TRUE

        if explain:
            print(self.explain(facts))

        return facts[self.consequent]

    # Return a string that explains the reason why this rule's value changed.
    def explain(self, facts: dict[str, Truth]):
        result = ''
        result += f'    {"GOAL" if self.is_goal else "RULE"} {self.consequent} is {facts[self.consequent]} because:'
        result += f'\n        it follows from {"NOT " if self.is_negated else ""}"{self.antecedent_string}" and:'

        for fact in self.antecedent.list_facts():
            if facts[fact] == Truth.UNKNOWN: continue
            result += f'\n            {fact} = {facts[fact]}'

        return result


# Represents a Boolean expression.
class BoolExpression:
    operator: Operator | None
    operand1: str | BoolExpression | None
    operand2: BoolExpression | None

    # Parse the expression.
    def __init__(self, expression: str):
        expression = expression.strip()
        self.expression = expression
        self.operator = None
        self.operand1 = None
        self.operand2 = None

        best_prec = Precedence.NONE
        best_i = -1
        best_op = Operator.NONE

        paren_count = 0

        for i, c in enumerate(expression):
            c = expression[i]
            if c == '(':
                paren_count += 1
            elif c == ')':
                paren_count -= 1
            elif paren_count == 0:
                new_op, new_prec = parse_operator(c)
                if new_prec < best_prec:
                    best_prec = new_prec
                    best_i = i
                    best_op = new_op
        
        if best_i < 0:
            if expression[0] == '(':
                expression = expression.lstrip('(').rstrip(')')
                self.operator = Operator.PAREN
                self.operand1 = BoolExpression(expression)
                self.operand2 = None
                return
            else:
                self.operator = Operator.FACT
                self.operand1 = expression
                self.operand2 = None
                return  
        
        self.operator = best_op

        if best_op == Operator.NOT:
            self.operand1 = BoolExpression(expression.lstrip('!'))
        else:
            self.operand1 = BoolExpression(expression[:best_i])
            self.operand2 = BoolExpression(expression[best_i + 1:])

    # Return a set holding the facts that this expression needs for evaluation.
    def list_facts(self) -> set[str]:
        if self.operator == Operator.FACT:
            operand1 = cast(str, self.operand1)
            return {operand1}

        if self.operator == Operator.PAREN or self.operator == Operator.NOT:
            operand1 = cast(BoolExpression, self.operand1)
            return operand1.list_facts()

        if self.operator == Operator.AND or self.operator == Operator.OR:
            operand1 = cast(BoolExpression, self.operand1)
            operand2 = cast(BoolExpression, self.operand2)
            return operand1.list_facts().union(operand2.list_facts())
        
        return set()

    # Evaluate the expression given the current facts.
    def evaluate(self, facts: dict[str, Truth]):
        if self.operator == Operator.FACT:
            operand1 = cast(str, self.operand1)
            return facts[operand1]

        if self.operator == Operator.PAREN:
            operand1 = cast(BoolExpression, self.operand1)
            return operand1.evaluate(facts)

        if self.operator == Operator.NOT:
            operand1 = cast(BoolExpression, self.operand1)
            truth1 = operand1.evaluate(facts)
            if truth1 == Truth.TRUE:
                return Truth.FALSE
            if truth1 == Truth.FALSE:
                return Truth.TRUE
            return Truth.UNKNOWN
        
        operand1 = cast(BoolExpression, self.operand1)
        operand2 = cast(BoolExpression, self.operand2)
        truth1 = operand1.evaluate(facts)
        truth2 = operand2.evaluate(facts)

        if self.operator == Operator.AND:
            if truth1 == Truth.TRUE and truth2 == Truth.TRUE:
                return Truth.TRUE
            if truth1 == Truth.FALSE or truth2 == Truth.FALSE:
                return Truth.FALSE
            return Truth.UNKNOWN

        if self.operator == Operator.OR:
            if truth1 == Truth.TRUE or truth2 == Truth.TRUE:
                return Truth.TRUE
            if truth1 == Truth.FALSE and truth2 == Truth.FALSE:
                return Truth.FALSE
            return Truth.UNKNOWN

In [218]:
class Truthifier:
    def __init__(self, rule_strings):
        self.load_rules(rule_strings)
        self.load_facts()
        self.load_questions()
        self.goals_reached = []

    # Build the rules list.
    def load_rules(self, rule_strings):
        self.rules = []
        for rule_string in rule_strings:
            self.rules.append(Rule(rule_string))

    # Build the facts dictionary.
    def load_facts(self):
        self.facts = {}
        for rule in self.rules:
            for fact in rule.list_facts():
                self.facts[fact] = Truth.UNKNOWN

    # Build the questions list.
    # This includes all facts that are not consequents.
    def load_questions(self):
        # Build a list of consequents.
        consequents = set()
        for rule in self.rules:
            consequents.add(rule.consequent)

        # Make the list of questions.
        self.questions = []
        for rule in self.rules:
            for fact in rule.antecedent.list_facts():
                if fact not in consequents:
                    if fact not in self.questions:
                        self.questions.append(fact)

    # Display the questions.
    def dump_questions(self):
        print("\n*** QUESTIONS ***")
        for question in self.questions:
            print(f'    {question} = {self.facts[question]}')

    # Display the rules.
    def dump_rules(self):
        print("\n*** RULES ***")
        for rule in self.rules:
            print(f'    {rule.rule_string} = {self.facts[rule.consequent]}')

    # Display the facts.
    def dump_facts(self):
        print("\n*** FACTS ***")
        for key, value in self.facts.items():
            print(f'    {key}: {value}')

    # Ask questions until we reach a goal.
    def solve(self):
        EXPLAIN = True

        # Print instructions.
        print('Instructions:')
        print('    y = Yes')
        print('    n = No')
        print('    ? = Unknown')
        print('    d = Dump data')
        print('    q = Quit')
        print()

        # Ask questions.
        for question in self.questions:
            # Only ask about UNKNOWN questions.
            if self.facts[question] == Truth.UNKNOWN:
                # Ask about this question.
                while True:
                    answer = input(f'{question}? ').lower()
                    if answer == 'q':
                        print('Quitting')
                        return
                    elif answer == 'y':
                        self.facts[question] = Truth.TRUE
                        break
                    elif answer == 'n':
                        self.facts[question] = Truth.FALSE
                        break
                    elif answer == '?':
                        break
                    elif answer == 'd':
                        # Dump data.
                        self.dump_questions()
                        self.dump_rules()
                        self.dump_facts()
                        print()

                # Try solving again.
                self.evaluate_rules(EXPLAIN)

                # If we reached a goal, break out of the input loop.
                if len(self.goals_reached) > 0:
                    break

    # Evaluate UNKNOWN rules as long as something changes.
    def evaluate_rules(self, explain):
        had_change = True
        while had_change:
            # Assume nothing will change during this pass.
            had_change = False

            # Evaluate the rules.
            for rule in self.rules:
                # Only evaluate rules that are UNKNOWN.
                if rule.truth(self.facts) == Truth.UNKNOWN:
                    truth = rule.evaluate(self.facts, explain)
                    if truth != Truth.UNKNOWN:
                        # This rule now has a value.
                        self.facts[rule.consequent] = truth
                        had_change = True

                        # If this is a goal, and it's true,
                        # and it's not already in goals_reached,
                        # then add it to goals_reached.
                        if rule.is_goal and \
                                (truth == Truth.TRUE) and \
                                (rule not in self.goals_reached):
                            self.goals_reached.append(rule)

In [219]:
# The main program.

# Animals:
#       platypus
#       squirrel
#       coelacanth
#       alligator
#       giraffe
#       squid
#       tiger
#       tortoise

# Load the rule base.
rule_strings = [
    'has fur -> mammal',
    'mammal -> !reptile',
    'scaled & breathes water -> fish',
    'scaled & !breathes water -> reptile',
    'mammal & lays eggs -> $platypus',
    'mammal & looks like a mad scientist experiment -> $platypus',
    'mammal & arboreal -> $squirrel',
    'fish & !many arms -> $coelacanth',
    'reptile & !shelled -> $alligator',
    'mammal & ridiculously long neck -> $giraffe',
    'breathes water & many arms -> $squid',
    '(mammal & carnivore) | (mammal & striped) -> $tiger',
    'scaled & !breathes water & shelled -> $tortoise',
]

# Make the expert.
expert = Truthifier(rule_strings)

# Solve the system.
expert.solve()

# See if we have reached any goals.
if len(expert.goals_reached) == 0:
    # We're doomed!
    print('Sorry, I was unable to identify the animal.😔')
elif len(expert.goals_reached) == 1:
    # We reached one goal.
    rule_name = expert.goals_reached[0].consequent
    print(f'You are probably looking at {article(rule_name)} {rule_name}')
else:
    # We reached multiple goals.
    print(f'You are probably looking at one of the following animals:')
    for rule in expert.goals_reached:
        rule_name = rule.consequent
        print(f'    {article(rule_name)} {rule_name}')

Instructions:
    y = Yes
    n = No
    ? = Unknown
    d = Dump data
    q = Quit

    RULE mammal is TRUE because:
        it follows from "has fur" and:
            has fur = TRUE
    RULE reptile is FALSE because:
        it follows from NOT "mammal" and:
            mammal = TRUE
    RULE fish is UNKNOWN because:
        it follows from "scaled & breathes water" and:
    GOAL platypus is UNKNOWN because:
        it follows from "mammal & lays eggs" and:
            mammal = TRUE
    GOAL platypus is UNKNOWN because:
        it follows from "mammal & looks like a mad scientist experiment" and:
            mammal = TRUE
    GOAL squirrel is UNKNOWN because:
        it follows from "mammal & arboreal" and:
            mammal = TRUE
    GOAL coelacanth is UNKNOWN because:
        it follows from "fish & !many arms" and:
    GOAL alligator is UNKNOWN because:
        it follows from "reptile & !shelled" and:
            reptile = FALSE
    GOAL giraffe is UNKNOWN because:
        it f